In [1]:
from load_trajectories import get_timestamps, load_trajectories
# Using the NPL 2024a kernel
import os
import os.path

#for calculating the air temperature
from metpy.units import units
from metpy.calc import temperature_from_potential_temperature

#time
from datetime import timedelta

import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(


# Folder 1 (using one netcdf file)

In [2]:
dirpath = "/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200"
fn = "cm1out_only_upto35.nc"
newncfilepath = os.path.join(dirpath, fn)
print(newncfilepath)

/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200/cm1out_only_upto35.nc


In [3]:
#Opened the netcdf files into a Dataset
nc = xr.open_dataset(newncfilepath)
nc

<xarray.Dataset>
Dimensions:   (one: 1, time: 105, ni: 120, nip1: 121, nj: 120, njp1: 121,
               nk: 140, nkp1: 141)
Coordinates:
  * time      (time) timedelta64[ns] 01:26:00 01:27:00 ... 02:10:00.000488281
Dimensions without coordinates: one, ni, nip1, nj, njp1, nk, nkp1
Data variables: (12/43)
    f_cor     (one) float32 ...
    ztop      (one) float32 ...
    xh        (ni) float32 ...
    xf        (nip1) float32 ...
    yh        (nj) float32 ...
    yf        (njp1) float32 ...
    ...        ...
    out10     (time, nk, nj, ni) float32 ...
    out11     (time, nk, nj, ni) float32 ...
    out12     (time, nk, nj, ni) float32 ...
    out13     (time, nk, nj, ni) float32 ...
    out14     (time, nk, nj, ni) float32 ...
    tke       (time, nkp1, nj, ni) float32 ...
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [22]:
(nc['xf'].values) * 1000
#xf = x position of the edges
#yf = y position of the edges
#xh = x position of the midpoint
#yh = y position of the midpoint

array([    0.     ,   100.     ,   200.     ,   300.     ,   400.     ,
         500.     ,   600.     ,   700.00006,   800.     ,   900.00006,
        1000.     ,  1100.     ,  1200.     ,  1300.0001 ,  1400.0001 ,
        1500.0001 ,  1600.     ,  1700.     ,  1800.0001 ,  1900.0001 ,
        2000.     ,  2100.0002 ,  2200.     ,  2300.0002 ,  2400.     ,
        2500.     ,  2600.0002 ,  2700.     ,  2800.0002 ,  2900.     ,
        3000.0002 ,  3100.0002 ,  3200.     ,  3300.0002 ,  3400.     ,
        3500.0002 ,  3600.0002 ,  3700.0002 ,  3800.0002 ,  3900.     ,
        4000.     ,  4100.0005 ,  4200.0005 ,  4300.     ,  4400.     ,
        4500.     ,  4600.0005 ,  4700.0005 ,  4800.     ,  4900.     ,
        5000.     ,  5100.0005 ,  5200.0005 ,  5300.     ,  5400.     ,
        5500.0005 ,  5600.0005 ,  5700.0005 ,  5800.     ,  5900.     ,
        6000.0005 ,  6100.0005 ,  6200.0005 ,  6300.     ,  6400.     ,
        6500.0005 ,  6600.0005 ,  6700.0005 ,  6800.     ,  6900

In [4]:
#Creating a DataArray object of the time
# time_coords = (nc['time'].values) / 1e9

time_coords = nc['time'].values / np.timedelta64(1, 's')
time_coords.astype(np.int32)

array([5160, 5220, 5280, 5340, 5400, 5460, 5520, 5580, 5640, 5700, 5760,
       5820, 5880, 5940, 6000, 6060, 6120, 6180, 6240, 6300, 6360, 6420,
       6480, 6540, 6600, 6660, 6720, 6780, 6840, 6900, 6960, 7020, 7080,
       7140, 7200, 3660, 3720, 3780, 3840, 3900, 3960, 4020, 4080, 4140,
       4200, 4260, 4320, 4380, 4440, 4500, 4560, 4620, 4680, 4740, 4800,
       4860, 4920, 4980, 5040, 5100, 5160, 5220, 5280, 5340, 5400, 5460,
       5520, 5580, 5640, 5700, 5760, 5820, 5880, 5940, 6000, 6060, 6120,
       6180, 6240, 6300, 6360, 6420, 6480, 6540, 6600, 6660, 6720, 6780,
       6840, 6900, 6960, 7020, 7080, 7140, 7200, 7260, 7320, 7380, 7440,
       7500, 7560, 7620, 7680, 7740, 7800], dtype=int32)

In [4]:
#Selecting certain data variables
selected_vars = ['rh', 'th', 'prs', 'uinterp', 'vinterp', 'winterp', 'out8', 'out9', 'out10', 'out11', 'out12', 'out13', 'out14', 'deactrat']
selected_data = nc[selected_vars]
selected_data

<xarray.Dataset>
Dimensions:   (time: 105, nk: 140, nj: 120, ni: 120)
Coordinates:
  * time      (time) timedelta64[ns] 01:26:00 01:27:00 ... 02:10:00.000488281
Dimensions without coordinates: nk, nj, ni
Data variables: (12/14)
    rh        (time, nk, nj, ni) float32 ...
    th        (time, nk, nj, ni) float32 ...
    prs       (time, nk, nj, ni) float32 ...
    uinterp   (time, nk, nj, ni) float32 ...
    vinterp   (time, nk, nj, ni) float32 ...
    winterp   (time, nk, nj, ni) float32 ...
    ...        ...
    out10     (time, nk, nj, ni) float32 ...
    out11     (time, nk, nj, ni) float32 ...
    out12     (time, nk, nj, ni) float32 ...
    out13     (time, nk, nj, ni) float32 ...
    out14     (time, nk, nj, ni) float32 ...
    deactrat  (time, nk, nj, ni) float32 ...
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [5]:
# adding variable example
#concantenate into int
selected_data['time_seconds'] = nc['time'].values / np.timedelta64(1, 's')
selected_data['time_seconds'] = selected_data['time_seconds'].astype(np.int32)
selected_data
#selected_data['time_seconds'] = (nc['time'].values) / 1e9 
# selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, time_seconds: 105)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5280 5340 ... 7680 7740 7800
Dimensions without coordinates: nk, nj, ni
Data variables: (12/14)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 ...
    prs           (time, nk, nj, ni) float32 ...
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    out10         (time, nk, nj, ni) float32 ...
    out11         (time, nk, nj, ni) float32 ...
    out12         (time, nk, nj, ni) float32 ...
    out13         (time, nk, nj, ni) float32 ...
    out14         (time, nk, nj, ni) float32 ...
    deactrat      (time, nk, nj, ni) float32 ...
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [ ]:
# df = selected_data.to_dataframe()
# df

In [ ]:
# #Converting a data set to a dataframe & renaming the column names
# df = selected_data.to_dataframe()
# renamed_table = df.rename(columns={
#     'rh': 'Relative Humidity',
#     'th': 'Potential Temperature (K)',
#     'prs': 'Pressure (pa)',
#     'uinterp': 'U Interpolated to Scalar Points (Velocity Field)',
#     'vinterp': 'V Interpolated to Scalar Points (Velocity Field)',
#     'winterp': 'W Interpolated to Scalar Points (Velocity Field)',
#     'out8': 'out8: mean size in um (a)',
#     'out9': 'out9: mean size in um (c)',
#     'out10': 'out10: ice number mixing ratio (#/kg)',
#     'out11': 'out11: mean settling velocity (m/s)',
#     'out12': 'out12: mean ice density (kg/m^3)',
#     'out13': 'out13: standard deviation of size in um (a_std)',
#     'out14': 'out14: standard deviation of size in um (c_std)',
#     'deactrat': 'deactrat: mean deposition/sublimation rate (kg/kg/s)',
#     }
# )
# df_indexed = renamed_table.reset_index()
# df_indexed

# #.astype (convert float to int)

In [14]:
selected_data['prs']

<xarray.DataArray 'prs' (time: 105, nk: 140, nj: 120, ni: 120)>
array([[[[101377.91 , ..., 101377.87 ],
         ...,
         [101377.95 , ..., 101377.91 ]],

        ...,

        [[ 14848.887, ...,  14848.887],
         ...,
         [ 14848.887, ...,  14848.887]]],


       ...,


       [[[101373.78 , ..., 101373.81 ],
         ...,
         [101373.78 , ..., 101373.86 ]],

        ...,

        [[ 14840.043, ...,  14840.037],
         ...,
         [ 14840.043, ...,  14840.043]]]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 01:26:00 01:27:00 ... 02:10:00.000488281
Dimensions without coordinates: nk, nj, ni
Attributes:
    long_name:  pressure
    units:      Pa

In [ ]:
# #adding a column for temperature (calculating from potential temperature)
# potential_temp = selected_data['th'].values * units.kelvin
# pressure = (selected_data['prs'].values / 100) * units.hPa
# selected_data['air_temp'] = temperature_from_potential_temperature(pressure, potential_temp)
# selected_data

In [18]:
selected_data['air_temp'] = selected_data['th'] * (selected_data['prs'] / 101325) ** .286
selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, time_seconds: 105)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5280 5340 ... 7680 7740 7800
Dimensions without coordinates: nk, nj, ni
Data variables: (12/15)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 294.7 294.7 294.7 ... 366.8 366.8
    prs           (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    out11         (time, nk, nj, ni) float32 ...
    out12         (time, nk, nj, ni) float32 ...
    out13         (time, nk, nj, ni) float32 ...
    out14         (time, nk, nj, ni) float32 ...
    deactrat      (time, nk, nj, ni) float32 ...
    air_temp      (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [ ]:
#Checks if the velocity fields are different

#renamed_table['out8: mean size in um (a)'].values
#renamed_table['out9: mean size in um (c)'].values
np.unique(df_indexed['out10: ice number mixing ratio (#/kg)'].values)
# renamed_table['out11: mean settling velocity (m/s)'].values
# renamed_table['out12: mean ice density (kg/m^3)'].values
# renamed_table['out13: standard deviation of size in um (a_std)'].values
# renamed_table['out14: standard deviation of size in um (c_std)'].values

In [ ]:
#TIME
time = df_indexed['time'].values
df_indexed.set_index('time', inplace=True)
df_indexed

In [ ]:
#plotting time and air temperature
time = df_indexed['time'].values
air_temp = df_indexed['Air Temperature'].values

#plot
plt.plot(time, air_temp, label='Air Temperature')
plt.xlabel('Time')
plt.ylabel('Air Temperature (K/Pa)')
plt.title('Air Temperature vs Time')

# Folder 1 (another cdf file for comparison)

In [ ]:
path1 = '/glade/derecho/scratch/joko/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly'
fn1 = "cm1out.nc"
path1file = os.path.join(path1, fn1)

path2 = '/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200'
fn2 = "cm1out_only_upto35.nc"
path2file = os.path.join(path2, fn2)

ds1 = xr.open_dataset(path1file)
ds2 = xr.open_dataset(path2file)

In [ ]:
# get the intersection of two datasets, on time dimension
ds2 = ds2.drop_duplicates(dim='time', keep='first') # drop duplicate time steps, keep the first instance

In [ ]:
time_idx = np.isin(ds1.time, ds2.time) # get indices where timestamps of ds1 are in timestamps of ds2

In [ ]:
ds1 = ds1.sel(time=time_idx, drop=True) # get subset based on timestamps
ds2 = ds2.sortby('time') # sort by timestamps

if ds1['time'].equals(ds2['time']):
    print('timestamps match')
else:
    print('timestamps do not match')
var = 'out8' # variable to compare
if ds1[var].equals(ds2[var]):
    print(f'Data match for variable {var}')
else:
    print(f'Data do NOT match for variable {var}')